# Create custom length subtitles with AssemblyAI



While our SRT/VTT endpoints do allow you to customize the maximum number of characters per caption using the chars_per_caption URL parameter in your API requests, there are some use-cases that require a custom number of words in each subtitle.

In this Colab, we will demonstrate how to construct these subtitles yourself in Python!

## Step-by-Step Instructions

In [1]:
pip install -U assemblyai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00


Import the assemblyai package and set the API key.



In [2]:
import assemblyai as aai

aai.settings.api_key = "YOUR-API-KEY"

Create a Transcriber object.


In [3]:
transcriber = aai.Transcriber()


Use the Transcriber object's transcribe method and pass in the audio file's path as a parameter. The transcribe method saves the results of the transcription to the Transcriber object's transcript attribute.

In [ ]:
transcript = transcriber.transcribe("./my-audio.mp3")

Alternatively, you can pass in the URL of the publicly accessible audio file on the internet.



In [4]:
transcript = transcriber.transcribe("https://storage.googleapis.com/aai-docs-samples/espn.m4a")

Define a function that converts seconds to timecodes

In [5]:
def second_to_timecode(x: float) -> str:
    hour, x = divmod(x, 3600)
    minute, x = divmod(x, 60)
    second, x = divmod(x, 1)
    millisecond = int(x * 1000.)

    return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)

Define a function that iterates through the transcripts object to construct a list according to the number of words per subtitle

In [18]:
def generate_subtitles_by_word_count(words_per_line):
  output = []
  i = 0
  while i < len(transcript.words):
      output.append(str(i // words_per_line + 1))  # Adjusted subtitle index

      start_time = second_to_timecode(transcript.words[i].start/1000)

      # Check for out-of-bounds and adjust the end time accordingly
      if i + words_per_line - 1 < len(transcript.words):
          end_time = second_to_timecode(transcript.words[i + words_per_line - 1].end/1000)
      else:
          end_time = second_to_timecode(transcript.words[-1].end/1000)

      output.append("%s --> %s" % (start_time, end_time))

      # Concatenate the required number of words together
      words = " ".join([word.text for word in transcript.words[i:i+words_per_line]])
      output.append(words)
      output.append("")

      i += words_per_line

  return output

Generate your subtitle file

In [23]:
subs = generate_subtitles_by_word_count(1)
with open(f"{transcript.id}.srt", 'w') as o:
    final = '\n'.join(subs)
    o.write(final)

print("SRT file generated.")

SRT file generated.


# Conclusion
This is a useful script for those looking to customise the number of words in their captions instead of the number of characters!